In [177]:
import pandas as pd
import numpy as np
from sklearn import svm, datasets, preprocessing
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import scale

In [336]:
data_train = pd.read_csv("C:\Serega\Python\elections_usa96_train.csv")

In [337]:
X = data_train.loc[:,"popul":"income"]
y = data_train.loc[:,"vote"]

In [338]:
d1={"extLib":1, "Lib":2, "sliLib":3, "Mod":4, "sliCon":5, "Con":6, "extCon":7}
d2={"MS":1, "HSdrop":2, "HS":6, "Coll":8, "CCdeg":10, "BAdeg":15, "MAdeg":20}
def sas(q):
    i = 0
    w = pd.Series()
    for inc in q:
        inc = inc.replace("$", "")
        inc = inc.replace("inus", "")
        inc = inc.replace("lus", "")
        inc = inc.replace("K", "")
        inc = inc.split("-")
        if len(inc) == 2:
            inc = (float(inc[0])+float(inc[1]))/2
        elif (inc[0][-1]=="p"):
            inc = float(inc[0][:-1])*1.2            
        else:
            inc = float(inc[0][:-1])*0.8
        w = w.set_value(i, inc)
        i += 1
    return w

In [339]:
X = X.replace({"ClinLR": d1})
X = X.replace({"DoleLR": d1})
X = X.replace({"educ": d2})
v = sas(X["income"])
X["income"] = v
y = y.replace(["Dole","Clinton"],[1,0])

In [340]:
X = scale(X.as_matrix())
y = y.as_matrix()

In [341]:
parameters = {'kernel':['rbf'], 'C':np.arange(0.75,1.3,0.025)}
#parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

In [342]:
svr = svm.SVC()

In [343]:
clf = GridSearchCV(svr, param_grid = parameters, cv = 20)

In [344]:
clf.fit(X, y)

GridSearchCV(cv=20, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([ 0.75 ,  0.775,  0.8  ,  0.825,  0.85 ,  0.875,  0.9  ,  0.925,
        0.95 ,  0.975,  1.   ,  1.025,  1.05 ,  1.075,  1.1  ,  1.125,
        1.15 ,  1.175]), 'kernel': ['rbf']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [345]:
print(clf.best_params_, clf.best_score_)

{'C': 1.1750000000000003, 'kernel': 'rbf'} 0.771186440678


In [346]:
Xn = pd.read_csv("C:\Serega\Python\elections_usa96_test.csv")
Xn = Xn.replace({"ClinLR": d1})
Xn = Xn.replace({"DoleLR": d1})
Xn = Xn.replace({"educ": d2})
v = sas(Xn["income"])
Xn["income"] = v
Xn = scale(Xn)

In [347]:
t = clf.predict(Xn)
t = pd.Series(t).replace([1,0],["Dole","Clinton"])
z = np.arange(len(t))
d = {'vote' : t}
df = pd.DataFrame(d)

df.to_csv("C:\Serega\Python\elections_usa96_predict.csv")
df.head()

,vote
0,Dole
1,Clinton
2,Clinton
3,Clinton
4,Clinton
